In [1]:
from PyAFAR_GUI.infant_afar import infant_afar

2025-02-06 10:32:21.702580: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Matplotlib is building the font cache; this may take a moment.
I0000 00:00:1738834404.543547 1720254 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-20.6.4), renderer: Intel(R) Iris(TM) Plus Graphics 655
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


W0000 00:00:1738834404.548829 1723377 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


~ 3min

In [24]:
import pandas as pd 
import os

In [27]:
folder_path = "../data/CityInfant/BlackWhite"

# List of files to remove
to_remove = ['Baby face 1.bmp', 'baby face 2.bmp', 'baby face 3.bmp', 'G33NegBW(1).bmp']

# Get all files in the folder
files = os.listdir(folder_path)

# Loop through files
for file in files:
    file_path = os.path.join(folder_path, file)

    if file in to_remove:
        os.remove(file_path)  # Remove unwanted files
        print(f"Deleted: {file}")

    elif file.startswith("Copy of "):  # Check if it's a copy
        original_file = file.replace("Copy of ", "", 1)  # Get the original filename
        original_path = os.path.join(folder_path, original_file)

        if original_file in files:
            os.remove(file_path)  # Delete the copy
            print(f"Deleted duplicate: {file}")
        else:
            new_path = os.path.join(folder_path, original_file)
            os.rename(file_path, new_path)  # Rename the copy
            print(f"Renamed {file} to {original_file}")


Deleted: G33NegBW(1).bmp


In [14]:
# Full AU
# AUs=["au_1", "au_2", "au_3","au_4", "au_6", "au_7","au_9", "au_10", "au_12","au_14", "au_15", "au_17","au_20","au_23", "au_24", "au_28"]

In [16]:
results

{'Frame': [],
 'Pitch': [0.7506699202373603],
 'Yaw': [3.112263569289325],
 'Roll': [-0.016583124017347965],
 'Eye Aspect Ratio': [0.8782328276849487],
 'Mouth Aspect Ratio': [0.5015342260964427],
 'Occ_au_1': [0.45760752562121787],
 'Occ_au_2': [0.691762511990858],
 'Occ_au_3': [0.058158720511077025],
 'Occ_au_4': [0.010655957149146165],
 'Occ_au_6': [0.007330263188446232],
 'Occ_au_9': [1.6317088074534302e-05],
 'Occ_au_12': [0.0022234563490506156],
 'Occ_au_20': [0.0022702617858506597],
 'Occ_au_28': [0.06802746186516968],
 'x_0': [0.5039100646972656],
 'y_0': [0.7928742170333862],
 'z_0': [-0.10797590762376785],
 'x_1': [0.5022698044776917],
 'y_1': [0.6991538405418396],
 'z_1': [-0.2027391642332077],
 'x_2': [0.5028171539306641],
 'y_2': [0.7285330295562744],
 'z_2': [-0.10264939814805984],
 'x_3': [0.46889159083366394],
 'y_3': [0.6198390126228333],
 'z_3': [-0.15672102570533752],
 'x_4': [0.5019323825836182],
 'y_4': [0.6732291579246521],
 'z_4': [-0.2174420952796936],
 'x_5': [

In [17]:
image_2 = "../data/CityInfant/BlackWhite/B4NegBw.bmp"
image_3 = "../data/CityInfant/BlackWhite/B6PosBW.bmp" 
image_4 = "../data/CityInfant/BlackWhite/B11NegBW.bmp"

In [19]:
results_2 = infant_afar(image_2, AUs=["au_1", "au_2", "au_3","au_4", "au_6","au_9", "au_12","au_20","au_28"], GPU=True, max_frames=1000 )
results_2

../data/CityInfant/BlackWhite/B4NegBw.bmp
True
Length of video
-9223372036854775808


{'Frame': [],
 'Pitch': [-4.922006268982117],
 'Yaw': [8.610919784600874],
 'Roll': [0.1999210604442137],
 'Eye Aspect Ratio': [0.7197167954770829],
 'Mouth Aspect Ratio': [0.6084941774588261],
 'Occ_au_1': [0.4002913785300857],
 'Occ_au_2': [0.43268162014493955],
 'Occ_au_3': [0.771212147028296],
 'Occ_au_4': [0.932553541030622],
 'Occ_au_6': [0.6632173281232693],
 'Occ_au_9': [0.010629378627101322],
 'Occ_au_12': [0.4068378170531906],
 'Occ_au_20': [0.9337502948102056],
 'Occ_au_28': [0.038446829128154635],
 'x_0': [0.6114510297775269],
 'y_0': [0.720291018486023],
 'z_0': [-0.111423060297966],
 'x_1': [0.6190460324287415],
 'y_1': [0.6422704458236694],
 'z_1': [-0.19518016278743744],
 'x_2': [0.6132721304893494],
 'y_2': [0.6689841151237488],
 'z_2': [-0.11240745335817337],
 'x_3': [0.5838029384613037],
 'y_3': [0.5742937922477722],
 'z_3': [-0.1457502543926239],
 'x_4': [0.6195858120918274],
 'y_4': [0.6187362670898438],
 'z_4': [-0.20455028116703033],
 'x_5': [0.6172367930412292],

In [20]:
results_3 = infant_afar(image_3, AUs=["au_1", "au_2", "au_3","au_4", "au_6","au_9", "au_12","au_20","au_28"], GPU=True, max_frames=1000 )
results_3

../data/CityInfant/BlackWhite/B6PosBW.bmp
True
Length of video
-9223372036854775808


{'Frame': [],
 'Pitch': [3.7771254908156764],
 'Yaw': [9.824432385464744],
 'Roll': [0.47116456004738205],
 'Eye Aspect Ratio': [0.8138802724582131],
 'Mouth Aspect Ratio': [0.6051143365384988],
 'Occ_au_1': [0.5414515450611654],
 'Occ_au_2': [0.5124176322743892],
 'Occ_au_3': [0.39249433370485387],
 'Occ_au_4': [0.9039112725819551],
 'Occ_au_6': [0.9037378200317628],
 'Occ_au_9': [0.2296297405678573],
 'Occ_au_12': [0.7464255947012232],
 'Occ_au_20': [0.9816701793688073],
 'Occ_au_28': [0.0005144918413263123],
 'x_0': [0.6400002837181091],
 'y_0': [0.6570058465003967],
 'z_0': [-0.17229875922203064],
 'x_1': [0.671413242816925],
 'y_1': [0.5547864437103271],
 'z_1': [-0.22185176610946655],
 'x_2': [0.6501673460006714],
 'y_2': [0.5957216024398804],
 'z_2': [-0.14155347645282745],
 'x_3': [0.6355952620506287],
 'y_3': [0.48312026262283325],
 'z_3': [-0.13952895998954773],
 'x_4': [0.6769826412200928],
 'y_4': [0.5279719829559326],
 'z_4': [-0.22440245747566223],
 'x_5': [0.676140129566

In [21]:
results_4 = infant_afar(image_4, AUs=["au_1", "au_2", "au_3","au_4", "au_6","au_9", "au_12","au_20","au_28"], GPU=True, max_frames=1000 )
results_4

../data/CityInfant/BlackWhite/B11NegBW.bmp
True
Length of video
-9223372036854775808


{'Frame': [],
 'Pitch': [12.128154493700698],
 'Yaw': [5.277828952022786],
 'Roll': [0.11826714649290664],
 'Eye Aspect Ratio': [0.8068603154741525],
 'Mouth Aspect Ratio': [0.556038698405545],
 'Occ_au_1': [0.8557970008165945],
 'Occ_au_2': [0.360135893835981],
 'Occ_au_3': [0.9869246776205782],
 'Occ_au_4': [0.9860957162576048],
 'Occ_au_6': [0.9865218579830449],
 'Occ_au_9': [0.9231136472396394],
 'Occ_au_12': [0.14729030505624302],
 'Occ_au_20': [0.8840265683202307],
 'Occ_au_28': [0.07038644772293741],
 'x_0': [0.5648559331893921],
 'y_0': [0.6335786581039429],
 'z_0': [-0.1722191423177719],
 'x_1': [0.5690683722496033],
 'y_1': [0.5120090246200562],
 'z_1': [-0.21155451238155365],
 'x_2': [0.5617073178291321],
 'y_2': [0.5633589029312134],
 'z_2': [-0.13901659846305847],
 'x_3': [0.5222808718681335],
 'y_3': [0.43665802478790283],
 'z_3': [-0.12288112938404083],
 'x_4': [0.5676531195640564],
 'y_4': [0.481865257024765],
 'z_4': [-0.21237030625343323],
 'x_5': [0.5620549321174622]

In [23]:
image_5 = "../data/CityInfant/BlackWhite/Copy of B10NegBW.bmp"
results_5 = infant_afar(image_5, AUs=["au_1", "au_2", "au_3","au_4", "au_6","au_9", "au_12","au_20","au_28"], GPU=True, max_frames=1000 )
results_5

../data/CityInfant/BlackWhite/Copy of B10NegBW.bmp
True
Length of video
-9223372036854775808


{'Frame': [],
 'Pitch': [-4.966692724160096],
 'Yaw': [11.311085498484896],
 'Roll': [0.23747895912877093],
 'Eye Aspect Ratio': [0.7873624111894701],
 'Mouth Aspect Ratio': [0.5072985355256331],
 'Occ_au_1': [0.7891175265004422],
 'Occ_au_2': [0.42765592644634864],
 'Occ_au_3': [0.9476581128972398],
 'Occ_au_4': [0.988636593883647],
 'Occ_au_6': [0.9867906308752545],
 'Occ_au_9': [0.7752244221939212],
 'Occ_au_12': [0.09147318616208941],
 'Occ_au_20': [0.8856116489273821],
 'Occ_au_28': [0.010968629252033152],
 'x_0': [0.6697317361831665],
 'y_0': [0.8006153106689453],
 'z_0': [-0.09053714573383331],
 'x_1': [0.6909896731376648],
 'y_1': [0.7151777744293213],
 'z_1': [-0.20822031795978546],
 'x_2': [0.6703359484672546],
 'y_2': [0.7378509640693665],
 'z_2': [-0.10069363564252853],
 'x_3': [0.6382283568382263],
 'y_3': [0.6211375594139099],
 'z_3': [-0.18122462928295135],
 'x_4': [0.6924771070480347],
 'y_4': [0.6874746084213257],
 'z_4': [-0.2263064980506897],
 'x_5': [0.6849133968353